Laboratorium 4:  Docker i konteneryzacja modelu ML

In [1]:
%%writefile requirements.txt
flask
numpy
scikit-learn
gunicorn

Overwriting requirements.txt


In [2]:
%%writefile C:\Users\Kubzz\Desktop\pbs\s6\ntpd\4\Dockerfile
#Użycie lekkiego obrazu Pythona
FROM python:3.9-slim

#Ustawienie katalogu roboczego
WORKDIR /app

#Kopiowanie plików aplikacji
COPY . /app

#Instalacja zależności
RUN pip install --no-cache-dir -r requirements.txt

#Wystawienie portu
EXPOSE 5000

#Uruchomienie serwera Gunicorn
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:5000", "app:app"]

Overwriting C:\Users\Kubzz\Desktop\pbs\s6\ntpd\4\Dockerfile


In [3]:
%%writefile C:\Users\Kubzz\Desktop\pbs\s6\ntpd\4\app.py
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LinearRegression

app = Flask(__name__)

# Przykładowe dane do trenowania modelu ML
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "hello world"})

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    if "input" not in data:
        return jsonify({"error": "Brak wymaganej wartosci"}), 400
    try:
        input_value = np.array([[data["input"]]])
        prediction = model.predict(input_value).tolist()
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Overwriting C:\Users\Kubzz\Desktop\pbs\s6\ntpd\4\app.py


In [4]:
!curl http://localhost:5000

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server


In [5]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    12    0     0  100    12      0   2450 --:--:-- --:--:-- --:--:--  3000
curl: (52) Empty reply from server


In [6]:
!pip install redis


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/261.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/261.5 kB ? eta -:--:--
   ------ -------------------------------- 41.0/261.5 kB 653.6 kB/s eta 0:00:01
   -------------------------- ------------- 174.1/261.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 261.5/261.5 kB 2.0 MB/s eta 0:00:00


In [7]:
%%writefile C:\Users\Kubzz\Desktop\pbs\s6\ntpd\4\app.py
from flask import Flask, request, jsonify
import numpy as np
import redis
from sklearn.linear_model import LinearRegression

app = Flask(__name__)

# Połączenie z Redis (host "redis" zamiast "localhost", bo Docker używa nazw kontenerów)
redis_client = redis.Redis(host="redis", port=6379, decode_responses=True)

# Przykładowe dane do trenowania modelu ML
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "hello world"})

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    if "input" not in data:
        return jsonify({"error": "Brak wymaganej wartosci"}), 400
    try:
        input_value = np.array([[data["input"]]])
        prediction = model.predict(input_value).tolist()

        # Zapis wyniku do Redis
        redis_client.set("last_prediction", prediction[0])

        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route("/last", methods=["GET"])
def last_prediction():
    """ Pobiera ostatnią predykcję z Redis """
    last_pred = redis_client.get("last_prediction")
    if last_pred is None:
        return jsonify({"error": "Brak zapisanej predykcji"}), 404
    return jsonify({"last_prediction": float(last_pred)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Overwriting C:\Users\Kubzz\Desktop\pbs\s6\ntpd\4\app.py


In [8]:
!curl http://localhost:5000/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server


In [9]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    12    0     0  100    12      0   2681 --:--:-- --:--:-- --:--:--  3000
curl: (52) Empty reply from server


In [10]:
!curl http://localhost:5000/last

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server
